In [148]:
import requests
import json
import re
# Use the Zitong + bing interface to query the city
def city_lookup2(dic_city, comp_file):
    result = {}
    f = open(comp_file, 'r')
    stand_city = f.read()
    f.close()
    for i in dic_city.keys():
        result[i] = {}
        for j in dic_city[i]:
            result[i][j] = ''
            url = 'https://zi.tools/api/transliteration/search/' + j
            try:
                res = requests.get(url=url).text
                res = json.loads(res)
                aim = res['result']
                for k in aim:
                    if re.search(j, k[0][0], re.IGNORECASE):
                        if k[0][2] in stand_city:
                            result[i][j] = k[0][2]
                            break
                mix = j
                if result[i][j] != '':
                    mix = result[i][j]
            except:
                mix = j
            url = 'https://cn.bing.com/api/v6/Places/AutoSuggest?q=' + mix.replace("'", '') + '&appid=D41D8CD98F00B204E9800998ECF8427E1FBE79C2&structuredaddress=true&setmkt=zh-CN&setlang=zh-CN'
            try:
                content = json.loads(requests.get(url = url).text)
            except:
                continue
            if 'value' not in content:
                continue
            for res in content['value']:
                if 'address' not in res or 'addressRegion' not in res['address']:
                    continue
                if res['address']['addressRegion'] not in i and i not in res['address']['addressRegion']:\
                    continue
                if res['_type'] == 'Place':
                    if 'addressSubregion' not in res['address']:
                        result[i][j] = res['name']
                        break
                    else:
                        result[i][j] = res['address']['addressSubregion']
                        break
                else:
                    if 'addressSubregion' not in res['address']:
                        continue
                    else:
                        result[i][j] = res['address']['addressSubregion']
                        break
    return result

In [109]:
import requests
import json
# Use the interface of bing map to query Chinese cities
def city_lookup(dic_city):
    result = {}
    for i in dic_city.keys():
        result[i] = {}
        cities = dic_city[i]
        for aim in cities:
            url = 'https://cn.bing.com/api/v6/Places/AutoSuggest?q=' + aim.replace("'", '') + '&appid=D41D8CD98F00B204E9800998ECF8427E1FBE79C2&structuredaddress=true&setmkt=zh-CN&setlang=zh-CN'
            content = json.loads(requests.get(url = url).text)
            result[i][aim] = ''
            if 'value' not in content:
                continue
            for res in content['value']:
                if 'address' not in res or 'addressRegion' not in res['address']:
                    continue
                if res['address']['addressRegion'] not in i and i not in res['address']['addressRegion']:\
                    continue
                if res['_type'] == 'Place':
                    if 'addressSubregion' not in res['address']:
                        result[i][aim] = res['name']
                        break
                    else:
                        result[i][aim] = res['address']['addressSubregion']
                        break
                else:
                    if 'addressSubregion' not in res['address']:
                        continue
                    else:
                        result[i][aim] = res['address']['addressSubregion']
                        break
    return result